In [ ]:
#Import dependencies
import os
import pandas as pd
import numpy as np

#Import API key
from config import api_key,secret_api

#API
import requests
import json
import rauth
from pprint import pprint

#Goodreads API library
import goodreads
from goodreads import client

#Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

#Hierarchical data
from collections import defaultdict

# Original CSV data 

### Read CSV into dataframe

In [ ]:
csv = "Resources/Data/book_data.csv"
book_data_df = pd.read_csv(csv)

book_data_df.dtypes

In [ ]:
#Remove decimals
book_data_df["Volume"] = book_data_df["Volume"].astype(str).str.replace(".0","",regex=False)
book_data_df["Publication_date"] = book_data_df["Publication_date"].astype(str).str.replace(".0","",regex=False)
book_data_df["year_read"] = book_data_df["year_read"].astype(str).str.replace(".0","",regex=False)

#Rename columns
rename_bd_df = book_data_df.rename(columns={"Title":"book_title",
                                            "Series":"series_name", 
                                            "Volume":"volume", 
                                            "Author":"author_name",
                                            "Language":"language",
                                            "Fiction/Non":"fiction_non",
                                            "Genre":"genre",
                                            "Pgs":"number_of_pages", 
                                            "Publication_date":"date_of_publication"})

In [ ]:
#Select relevant columns
final_book_df = rename_bd_df[["book_title",
                                  "series_name",
                                  "author_name",
                                  "language",
                                  "fiction_non",
                                  "genre", 
                                  "number_of_pages",
                                  "year_read"]]

final_book_df.head()

In [ ]:
#Generate HTML table
html_books= final_book_df.to_html("Resources/Data/html_books_table.html")

# Request information from Goodreads API

In [ ]:
#Create Goodreads client instance
gc = client.GoodreadsClient(api_key,secret_api)

### Author hometown API request

In [ ]:
#Prepare list of authors for API and unique author table
authors = rename_bd_df["author_name"]
single_authors = authors.drop_duplicates(keep="first")
index_authors = single_authors.reset_index(drop=True)

In [ ]:
#Empty list to hold API data 
hometowns = []
works_count = []

#Loop for iterative API requests
for author in range(len(index_authors)):
    #Create a print log of each author
    print(f"Retrieving hometown{author} | {index_authors[author]}")
    
    #Hometown and number of works request
    author = gc.find_author(index_authors[author])
    cities = author.hometown
    works = author.works_count
    
    #Save to list
    hometowns.append(cities)
    works_count.append(works)

In [ ]:
#Create author and hometown dataframe
hometown_df = pd.DataFrame({"author_name":index_authors, "hometown":hometowns, "books_written":works_count})
drop_hometown = hometown_df.drop_duplicates(subset="author_name",keep="first")
index_hometown = drop_hometown.reset_index(drop=True).copy()

index_hometown

In [ ]:
#Look for missing values in the data 
index_hometown.isna().any()

In [ ]:
#Remove authors with missing hometown values 
final_authors_df = index_hometown.dropna(how="any")

final_authors_df

# SQL and Flask API

In [ ]:
#Create engine and connection to database
connection = "postgres:postgres@localhost:5432/books"
engine = create_engine(f'postgresql://{connection}')

In [ ]:
#Check for tables
engine.table_names()

In [ ]:
#Load Author dataframe into database
index_authors.to_sql("authors", con=engine, if_exists="append", index=False)

In [ ]:
#Load Hometown dataframe into database
final_authors_df.to_sql("hometowns", con=engine, if_exists="append", index=False)

In [ ]:
#Load Book dataframe into database
final_book_df.to_sql("books", con=engine, if_exists="append", index=False)

In [ ]:
#Confirm data has been added to the authors database
pd.read_sql_query('SELECT * FROM authors', con=engine).head()

In [ ]:
#Confirm data has been added to the hometowns database
pd.read_sql_query('SELECT * FROM hometowns', con=engine).head()

In [ ]:
#Confirm data has been added to the books database
pd.read_sql_query('SELECT * FROM books', con=engine).head()

# Sunburst Plot

### Hierarchy & JSON Testing

In [ ]:
#Select relevant columns
multi_index_df = final_book_df[["fiction_non","genre","author_name","book_title"]]
multi_index_df

In [ ]:
#Create multi-index dictionary from dataframe
pd.MultiIndex.from_frame(multi_index_df, names=["fiction_non","genre","author_name","book_title"])

In [ ]:
#Export dictionary to JSON
multi_index_df.to_json(orient='index')

In [ ]:
#Export dictionary to JSON
multi_index_df.to_json("/Users/leishla/Desktop/Bootcamp/Projects/02-Project/betterread/static/js/sunburst/hierarchy_index.json",orient='index')

In [ ]:
#Second attempt at data hierarchy
dict_df = final_book_df[["genre","author_name","series_name","book_title"]]
d = defaultdict(lambda: defaultdict(list))

for row in dict_df.itertuples():
    d[row[1]][row[2]].append(row[4])

d

In [ ]:
#Export nested dictionary to JSON
d.to_json("/Users/leishla/Desktop/Bootcamp/Projects/02-Project/betterread/static/js/sunburst/hierarchy_index.json")

### Parent and label tuples

In [ ]:
#Create dataframe with relevant columns
pair_df = final_book_df[["genre","author_name","series_name","book_title"]]

#Create empty column for genre tuple
pair_df["parent"] = ""

parent_df = pair_df[["parent","genre","author_name","series_name","book_title"]]

parent_df

## Create tuples

#### Empty string and genre tuple

In [ ]:
#Select relevant columns
g_df = parent_df[["parent","genre"]]

#Drop duplicate genres
genre_unique = g_df.drop_duplicates(keep="first")

#Create unique genre tuples
genre = [tuple(r) for r in genre_unique[["parent", "genre"]].to_numpy()]

#### Genre and author tuple

In [ ]:
#Select relevant columns
a_df = parent_df[["genre","author_name"]]

#Drop missing values from series column
author_drop = a_df.dropna(how="any")

#Drop duplicate genre and author pairs
author_unique = author_drop.drop_duplicates(keep="first")

#Create unique genre and author tuples
author = [tuple(r) for r in author_unique[["genre", "author_name"]].to_numpy()]

#### Author and series tuple

In [ ]:
#Select relevant columns
s_df = parent_df[["author_name","series_name"]]

#Drop missing values from series column
series_df = s_df.dropna(how="any")

#Drop duplicate author and series pairs
series_unique = series_df.drop_duplicates(keep="first")

#Create tuples with authors and available series names
series = [tuple(r) for r in series_unique[["author_name", "series_name"]].to_numpy()]

#### Book title and series tuple

In [ ]:
#Identify titles that do not belong to a series
series_query_df = parent_df.query("series_name != series_name")

#Drop series column
missing_series_df = series_query_df.drop(columns="series_name")

#Author and book title (without series) tuple
book_title = [tuple(r) for r in missing_series_df[["author_name", "book_title"]].to_numpy()]

#Available series and title tuple
book_series = [tuple(r) for r in series_df[["series_name", "book_title"]].to_numpy()]

### Join tuples into single list

In [ ]:
#Empty list
sunburst = []

#Add genre
sunburst.append(genre)

#Add author
sunburst.append(author)

#Add series
sunburst.append(series)

#Add titles with series
sunburst.append(book_title)

#Add titles withou series
sunburst.append(book_series)

sunburst

In [ ]:
#Write tuples to file
with open("tuples.txt", "w") as output:
    output.write(str(sunburst))

# Stacked Bar chart

In [ ]:
#Select relevant categories
bar_chart = final_book_df[["book_title","genre","year_read"]]

#Group data by year
year_group = bar_chart.groupby(["year_read","genre"], as_index=False)

year_count = year_group.count()

year_count.to_json("/Users/leishla/Desktop/Bootcamp/Projects/02-Project/betterread/templates/year_count.json",orient="index")

In [ ]:
#Load Book dataframe into database
year_count.to_sql("bardata", con=engine, if_exists="append", index=False)

In [ ]:
pd.read_sql_query('SELECT * FROM bardata', con=engine).head()

In [ ]:
bookc_df = final_book_df.groupby("year_read").count()
bookc_df